## Задача
Проанализируйте тест между группами 0 и 3 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше по сравнению с обычным CTR?

Проанализируйте тест между группами 1 и 2 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше по сравнению с обычным CTR?

Данные берём в том же диапазоне, в котором проводился АБ-тест.
При выполнении задания важно обосновать и аргументировать ваши выводы.

In [ ]:
import pandahouse as pandahouse
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import poisson
from scipy import stats
import matplotlib.pyplot as plt

#подключение к clickhouse
connection = {
    'host': 'https://clickhouse.lab.karpov.courses/',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20240620'
}

q = """
select exp_group,
        user_id,
        sum(action = 'like') as likes,
        sum(action = 'view') as views,
        likes/views as ctr
from {db}.feed_actions
where toDate(time) >= '2024-05-31' and toDate(time) <= '2024-06-06'
    and exp_group in (0,3)
group by exp_group, user_id
"""

df = pandahouse.read_clickhouse(q, connection=connection)
    


In [25]:
# Считаем общий CTR в контрольной группе  𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙=𝑠𝑢𝑚(𝑙𝑖𝑘𝑒𝑠)/𝑠𝑢𝑚(𝑣𝑖𝑒𝑤𝑠)
general_ctr = df[df.exp_group == 0].likes.sum()/df[df.exp_group == 0].views.sum()

# Посчитаем в обеих группах поюзерную метрику  𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠=𝑙𝑖𝑘𝑒𝑠−𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙∗𝑣𝑖𝑒𝑤𝑠
linearized_likes_0 = df[df.exp_group == 0].likes - general_ctr*df[df.exp_group == 0].views
linearized_likes_3 = df[df.exp_group == 3].likes - general_ctr*df[df.exp_group == 3].views

#T-тест
ttest1 = stats.ttest_ind(linearized_likes_0,
                linearized_likes_3,
                equal_var=False)

ttest2 = stats.ttest_ind(df[df.exp_group == 0].ctr,
                df[df.exp_group == 3].ctr,
                equal_var=False)

print(ttest1)
print(ttest2)

Ttest_indResult(statistic=-16.186230032932844, pvalue=1.4918137745326139e-58)
Ttest_indResult(statistic=-13.935320516755773, pvalue=6.216047483062228e-44)


Проанализируйте тест между группами 0 и 3 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше по сравнению с обычным CTR?

Отличие есть, на несколько порядков. p-value стало меньше. 

In [2]:
import pandahouse as pandahouse
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import poisson
from scipy import stats
import matplotlib.pyplot as plt

#подключение к clickhouse
connection = {
    'host': 'https://clickhouse.lab.karpov.courses/',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20240620'
}

q = """
select exp_group,
        user_id,
        sum(action = 'like') as likes,
        sum(action = 'view') as views,
        likes/views as ctr
from {db}.feed_actions
where toDate(time) >= '2024-05-31' and toDate(time) <= '2024-06-06'
    and exp_group in (1,2)
group by exp_group, user_id
"""

df = pandahouse.read_clickhouse(q, connection=connection)
    
# Считаем общий CTR в контрольной группе  𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙=𝑠𝑢𝑚(𝑙𝑖𝑘𝑒𝑠)/𝑠𝑢𝑚(𝑣𝑖𝑒𝑤𝑠)
general_ctr = df[df.exp_group == 1].likes.sum()/df[df.exp_group == 1].views.sum()

# Посчитаем в обеих группах поюзерную метрику  𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠=𝑙𝑖𝑘𝑒𝑠−𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙∗𝑣𝑖𝑒𝑤𝑠
linearized_likes_1 = df[df.exp_group == 1].likes - general_ctr*df[df.exp_group == 1].views
linearized_likes_2 = df[df.exp_group == 2].likes - general_ctr*df[df.exp_group == 2].views

ttest3 = stats.ttest_ind(linearized_likes_1,
                linearized_likes_2,
                equal_var=False)

ttest4 = stats.ttest_ind(df[df.exp_group == 1].ctr,
                df[df.exp_group == 2].ctr,
                equal_var=False)

print(ttest3)
print(ttest4)



Ttest_indResult(statistic=5.93637710193448, pvalue=2.9805064038668164e-09)
Ttest_indResult(statistic=0.4051491913112757, pvalue=0.685373331140751)


Проанализируйте тест между группами 1 и 2 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше по сравнению с обычным CTR?

Отличие есть в результатах теста. Как ранее было выяснено, у 2 группы ctr ненормально распределены данные, а линерализация как раз их приводит к нормальному распределению, поэтому p-value в этом случае отличиается от p-value при расчете 'сырых'  данных.